# Modify Inflow data

**Date:** 2026-01-19

**Authors:** Maximilian Manaila

**Summary:** This notebook provides tools to load, modify and save inflow input (Simstrat-like `Qin.dat`, `Sin.dat` or `Tin.dat`)

## Before You Start:

**No Python knowledge needed!** Just follow these steps:
1. Run each cell in order (click the ▶️ button or press Shift+Enter)
2. Look for the sections marked "👉 CUSTOMIZE" to modify parameters

## Content:

This notebook helps you modify the inflow input. You can:

1. **Load and display inflow data** Using data downloaded from https://www.alplakes.eawag.ch/downloads
2. **Modify inflow data** Standard mathematical operations on data as well as adding, removing and changing the order of inflow
3. **Save modified inflow data** To new or existing file

## Available Functions:

- `deep, surf, deep_order, surf_order, units = load_inflow_data(path, file, start='file_start', stop='file_stop', load_units=True)` — Load downloaded deep and surface inflows and save their ordering and optionally load units of the inflow
- `save_inflow_data(deep, surf, deep_order, surf_order, path, file, copy=True)` — Save modified inflow data to a new or a existing file
- `change_depth(data, col_index, depth)` — Change depth of one inflow
- `add_inflow(data, data_order, col_index=-1, values=0, depth=0)` — Add one inflow
- `remove_inflow(data, data_order, col_index=-1)` — Remove one inflow
- `change_order(data, data_order, new_col_index)` — Change order of inflows

---

## STEP 1: Import Required Libraries
## ESSENTIAL STEP

Here you import the python libraries and custom functions needed for the following steps.

Simply run this cell once. You don't need to change anything here.

In [1]:
# Import standard Python libraries for data handling
import os               # Used to work with files and folders
import numpy as np      # Used for mathematical operations and working with arrays of numbers
import pandas as pd     # Used to read and modify data


# Import custom functions specifically designed for Alplakes data
# These functions do the heavy lifting for fetching and modifying data
from Modify_input_functions import load_inflow_data, save_inflow_data, change_depth, add_inflow, remove_inflow, change_order

print("✓ All libraries loaded successfully!")
print("You're ready to start modifying inflow data.")

✓ All libraries loaded successfully!
You're ready to start modifying inflow data.


# Step 2
## Load local data

Here you load the inflow data from your locally saved file and display the units of the inflow data as well as the data to edit.

There are two types of inflows:
- `Deep Inflows`: Depth is fixed relative to bottom of lake, saved as `deep`
- `Surface Inflows`: Depth is fixed relative to surface of lake, saved as `surf`

Accordingly, two inflow tables are shown.

In both tables the first column is the `Datetime` and the following columns are the inflows with their depth and the inflow index (in brackets) in the title. 

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `local_path`: The path to the forcing data file
- `file_name`: The name of the forcing data file
  - `Qin.dat` for the water inflow
  - `Sin.dat` for the salinity inflow
  - `Tin.dat` for the temperature inflow
- `start`: From when on to edit forcing data
  - date and time in format 'YYYYMMDDHHMM' (Year-Month-Day-Hour-Minute)
  - example: '202510231500' = October 23, 2025 at 15:00 (3:00 PM)
  - default: 'file_start' -> start at beginning of file
- `stop`: Until when to edit forcing data
  - same format as `start`
  - default: 'file_stop' -> stop at end of file
- `load_units` : Whether to load the units from the file
    - `True` to load the units (default)
    - `False` to not load the units, returns an empty DataFrame to `units`

In [4]:
local_path = 'Biel'                                    # 👉 CUSTOMIZE path to the local file
file_name = 'Qin.dat'                                       # 👉 CUSTOMIZE name of the local file

deep, surf, deep_order, surf_order, units = load_inflow_data(
    path = local_path,                                      # input path to the local file
    file = file_name,                                       # input name of the local file
    start = '202306010000',                                 # 👉 CUSTOMIZE start of model output YYYYMMDDHHMM, or file_start
    stop = 'file_stop',                                     # 👉 CUSTOMIZE end of model output YYYYMMDDHHMM, or file_stop
    load_units = True                                       # 👉 CUSTOMIZE whether to load units or not
)

display(units)                                              # display the units
print('Deep Inflows')
display(deep.round(4))                                      # display the deep inflow data filtered to the date range
print('Surface Inflows')
display(surf.round(4))                                      # display the surface inflow data filtered to the date range

Loading data from local file: Biel\Qin.dat
✓ Loaded 3 deep inflows with 721 rows from Biel\Qin.dat
No surface inflow
✓ Filtered to date range: 721 rows between 2023-06-01 and 2023-07-01


,Depth,Deep Inflows,Surface Inflows
Unit,m,m3/s,m2/s


Deep Inflows


,Datetime,0.0 (1),0.0 (2),0.0 (3)
0,2023-06-01 00:00:00,238.20,1.78,0.00
1,2023-06-01 01:00:02,213.77,1.75,0.00
2,2023-06-01 01:59:57,206.80,1.70,0.00
3,2023-06-01 03:00:00,192.00,1.70,39.35
4,2023-06-01 04:00:02,174.70,1.70,0.00
...,...,...,...,...
716,2023-06-30 19:59:57,256.67,1.00,0.00
717,2023-06-30 21:00:00,259.10,1.03,0.00
718,2023-06-30 22:00:02,254.75,1.00,0.00
719,2023-06-30 22:59:57,266.83,1.00,0.00


Surface Inflows


,Datetime
0,2023-06-01 00:00:00
1,2023-06-01 01:00:02
2,2023-06-01 01:59:57
3,2023-06-01 03:00:00
4,2023-06-01 04:00:02
...,...
716,2023-06-30 19:59:57
717,2023-06-30 21:00:00
718,2023-06-30 22:00:02
719,2023-06-30 22:59:57


# Step 3
## Modify deep inflow data

This step presents various way to modify the deep inflow data.

You can modify the data in the following ways:
- `Add a constant`
- `Multiply by a constant`
- `Set to a constant`
- `Add a linear trend`
- `Change depth`
- `Add an inflow`
- `Remove an inflow`
- `Change order of inflows`

### Add constant

Here you add a constant to any deep inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `constant`: the constant to add
    - any real number (can also be negative and/or a fraction)
    - inflows should not become negative

In [5]:
index = 3                                        # 👉 CUSTOMIZE the inflow to operate on
constant = 1                                   # 👉 CUSTOMIZE the constant

deep.iloc[:, index] += constant                  # modify the data
if np.any(deep.iloc[:, index].values < 0):       # check for negative inflows
    deep.iloc[:, index] -= constant              # undo the modification in case there are any negative inflows
    print(f"Negative values for inflow ({index}) — no modification")
else:
    print(f"✓ Inflow ({index}) modified")
    display(deep.round(4))                       # display the modified data

✓ Inflow (3) modified


,Datetime,0.0 (1),0.0 (2),0.0 (3)
0,2023-06-01 00:00:00,238.20,1.78,1.00
1,2023-06-01 01:00:02,213.77,1.75,1.00
2,2023-06-01 01:59:57,206.80,1.70,1.00
3,2023-06-01 03:00:00,192.00,1.70,40.35
4,2023-06-01 04:00:02,174.70,1.70,1.00
...,...,...,...,...
716,2023-06-30 19:59:57,256.67,1.00,1.00
717,2023-06-30 21:00:00,259.10,1.03,1.00
718,2023-06-30 22:00:02,254.75,1.00,1.00
719,2023-06-30 22:59:57,266.83,1.00,1.00


### Multiply by constant

Here you multiply any deep inflow by a constant.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `constant`: the constant to multiply by
    - any non-negative real number (can also be a fraction)

In [6]:
index = 1                                        # 👉 CUSTOMIZE the inflow to operate on
constant = 1.5                                   # 👉 CUSTOMIZE the constant

if constant >= 0:                                # check that constant is non-negative
    deep.iloc[:, index] *= constant              # modify the data
    print(f"✓ Inflow ({index}) modified")
    display(deep.round(4))                       # display the modified data
else:
    print(f"Negative values for inflow ({index}) — no modification")

✓ Inflow (1) modified


,Datetime,0.0 (1),0.0 (2),0.0 (3)
0,2023-06-01 00:00:00,357.300,1.78,1.00
1,2023-06-01 01:00:02,320.655,1.75,1.00
2,2023-06-01 01:59:57,310.200,1.70,1.00
3,2023-06-01 03:00:00,288.000,1.70,40.35
4,2023-06-01 04:00:02,262.050,1.70,1.00
...,...,...,...,...
716,2023-06-30 19:59:57,385.005,1.00,1.00
717,2023-06-30 21:00:00,388.650,1.03,1.00
718,2023-06-30 22:00:02,382.125,1.00,1.00
719,2023-06-30 22:59:57,400.245,1.00,1.00


### Set to constant

Here you set any chosen deep inflow to a constant in time.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `constant`: the constant value of the inflow
    - any non-negative real number (can also be a fraction)

In [7]:
index = 2                                                   # 👉 CUSTOMIZE the inflow to operate on
constant = 5                                                # 👉 CUSTOMIZE the constant

if constant >= 0:                                           # check that constant is non-negative
    deep.iloc[:, index] = np.full((len(deep)), constant)    # modify the data
    print(f"✓ Inflow ({index}) modified")
    display(deep.round(4))                                  # display the modified data
else:
    print(f"Negative values for inflow ({index}) — no modification")

✓ Inflow (2) modified


,Datetime,0.0 (1),0.0 (2),0.0 (3)
0,2023-06-01 00:00:00,357.300,5.0,1.00
1,2023-06-01 01:00:02,320.655,5.0,1.00
2,2023-06-01 01:59:57,310.200,5.0,1.00
3,2023-06-01 03:00:00,288.000,5.0,40.35
4,2023-06-01 04:00:02,262.050,5.0,1.00
...,...,...,...,...
716,2023-06-30 19:59:57,385.005,5.0,1.00
717,2023-06-30 21:00:00,388.650,5.0,1.00
718,2023-06-30 22:00:02,382.125,5.0,1.00
719,2023-06-30 22:59:57,400.245,5.0,1.00


### Add linear trend

Here you add a linear trend to any chosen deep inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `step`: the time of change
    - any positive integer
    - `1` corresponds to one hour
    - accordingly `24` corresponds to one day
    -  `8760` corresponds to one year
- `slope`: the amount of change per time of change
    - any real number (can also be negative and/or a fraction)
    - inflow `(index)` increases by `slope` every `step` hours
- `offset`: the offset to the trend
    - any real number (can also be negative and/or a fraction)
    - default is `0`

In [8]:
index = 2                                                  # 👉 CUSTOMIZE the inflow to operate on
step = 8760                                                  # 👉 CUSTOMIZE the time of change
slope = 1                                               # 👉 CUSTOMIZE the slope of the trend
offset = 0                                                 # 👉 CUSTOMIZE the offset of the trend

rate = slope / step                                        # the rate of change
val_save = deep.iloc[:, index]                             # save the current values
deep.iloc[:, index] = (deep.iloc[:, index] +               # modify the data
    np.linspace(0, rate * (len(deep) - 1), len(deep)) +    # add the linear increase
    offset)                                                # add the offset
if np.any(deep.iloc[:, index].values < 0):                 # check for negative inflows
    deep.iloc[:, index] = val_save                         # undo the modification in case there are any negative inflows
    print(f"Negative values for inflow ({index}) — no modification")
else:
    print(f"✓ Inflow ({index}) modified")
    display(deep.round(4))                                 # display the modified data

✓ Inflow (2) modified


,Datetime,0.0 (1),0.0 (2),0.0 (3)
0,2023-06-01 00:00:00,357.300,5.0000,1.00
1,2023-06-01 01:00:02,320.655,5.0001,1.00
2,2023-06-01 01:59:57,310.200,5.0002,1.00
3,2023-06-01 03:00:00,288.000,5.0003,40.35
4,2023-06-01 04:00:02,262.050,5.0005,1.00
...,...,...,...,...
716,2023-06-30 19:59:57,385.005,5.0817,1.00
717,2023-06-30 21:00:00,388.650,5.0818,1.00
718,2023-06-30 22:00:02,382.125,5.0820,1.00
719,2023-06-30 22:59:57,400.245,5.0821,1.00


### Change depth

Here you change the depth of any chosen deep inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `depth`: the new depth
    - any real number (can also be negative and/or a fraction)

In [10]:
index = 1                                 # 👉 CUSTOMIZE the inflow to operate on
depth = -7                                # 👉 CUSTOMIZE the constant

deep = change_depth(deep, index, depth)   # modify the data
display(deep.round(4))                    # display the modified data

✓ Modified depth of inflow (1)


,Datetime,-7.0 (1),0.0 (2),0.0 (3)
0,2023-06-01 00:00:00,357.300,5.0000,1.00
1,2023-06-01 01:00:02,320.655,5.0001,1.00
2,2023-06-01 01:59:57,310.200,5.0002,1.00
3,2023-06-01 03:00:00,288.000,5.0003,40.35
4,2023-06-01 04:00:02,262.050,5.0005,1.00
...,...,...,...,...
716,2023-06-30 19:59:57,385.005,5.0817,1.00
717,2023-06-30 21:00:00,388.650,5.0818,1.00
718,2023-06-30 22:00:02,382.125,5.0820,1.00
719,2023-06-30 22:59:57,400.245,5.0821,1.00


### Add Inflow

Here you add a deep inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the new inflow
    - any positive integer not larger than the amount of existing deep inflows plus one, or `-1`
    - default is `-1` to append as the last inflow
- `values`: the values of the new inflow, can be:
    - any non-negative real number (can also be a fraction): to set a constant value in time
    - 1D array of non-negative real numbers (can also be a fraction) of the same length as the other inflows: for a varying inflow
    - default is `0`, constant in time
- `depth` : the depth of the new inflow
    - any real number
    - default is `0`

In [11]:
index = 3                                        # 👉 CUSTOMIZE the position of the new inflow
values = 3                                       # 👉 CUSTOMIZE the value of the new inflow
depth = 10                                        # 👉 CUSTOMIZE the depth of the new inflow

deep, deep_order = add_inflow(                   # modify the data
    deep, deep_order, index, values, depth)      # keep track of the modifications with deep_order
display(deep.round(4))                           # display the modified data

✓ Added inflow at (3)


,Datetime,-7.0 (1),0.0 (2),10.0 (3),0.0 (4)
0,2023-06-01 00:00:00,357.300,5.0000,3.0,1.00
1,2023-06-01 01:00:02,320.655,5.0001,3.0,1.00
2,2023-06-01 01:59:57,310.200,5.0002,3.0,1.00
3,2023-06-01 03:00:00,288.000,5.0003,3.0,40.35
4,2023-06-01 04:00:02,262.050,5.0005,3.0,1.00
...,...,...,...,...,...
716,2023-06-30 19:59:57,385.005,5.0817,3.0,1.00
717,2023-06-30 21:00:00,388.650,5.0818,3.0,1.00
718,2023-06-30 22:00:02,382.125,5.0820,3.0,1.00
719,2023-06-30 22:59:57,400.245,5.0821,3.0,1.00


### Remove Inflow

Here you remove a deep inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to remove
    - any integer with an existing deep inflow, or `-1`
    - default is `-1` to remove the last inflow

In [9]:
index = 3                                        # 👉 CUSTOMIZE the inflow to operate on

deep, deep_order = remove_inflow(                # modify the data
    deep, deep_order, index)                     # keep track of the modifications with deep_order
display(deep.round(4))                           # display the modified data

✓ Removed inflow (3)


,Datetime,0.0 (1),0.0 (2),0.0 (3),0.0 (4),-2.0 (5)
0,2023-06-01 00:00:00,316.45,1.0000,3.0,0.100,0.63
1,2023-06-01 01:00:02,302.05,1.0014,3.0,0.100,0.63
2,2023-06-01 01:59:57,292.03,1.0028,3.0,0.100,0.64
3,2023-06-01 03:00:00,299.05,1.0042,3.0,0.100,0.64
4,2023-06-01 04:00:02,294.03,1.0056,3.0,0.100,0.66
...,...,...,...,...,...,...
716,2023-06-30 19:59:57,380.29,1.9944,3.0,0.138,0.60
717,2023-06-30 21:00:00,403.23,1.9958,3.0,0.118,0.67
718,2023-06-30 22:00:02,406.66,1.9972,3.0,0.118,0.60
719,2023-06-30 22:59:57,404.31,1.9986,3.0,0.156,0.60


### Change order

Here you change the order of the deep inflows.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `order` : the new positions of the inflows
    - tuple with as many elements as existing deep inflows
    - example: `[5, 2, 3, 4, 1]`: inflows (1) and (5) change position

In [10]:
order = [5, 2, 3, 4, 1]                          # 👉 CUSTOMIZE the new order

deep, deep_order = change_order(                 # modify the data
    deep, deep_order, order)                     # keep track of the modifications with deep_order
display(deep.round(4))                           # display the modified data

✓ Changed inflow order


,Datetime,-2.0 (1),0.0 (2),0.0 (3),0.0 (4),0.0 (5)
0,2023-06-01 00:00:00,0.63,1.0000,3.0,0.100,316.45
1,2023-06-01 01:00:02,0.63,1.0014,3.0,0.100,302.05
2,2023-06-01 01:59:57,0.64,1.0028,3.0,0.100,292.03
3,2023-06-01 03:00:00,0.64,1.0042,3.0,0.100,299.05
4,2023-06-01 04:00:02,0.66,1.0056,3.0,0.100,294.03
...,...,...,...,...,...,...
716,2023-06-30 19:59:57,0.60,1.9944,3.0,0.138,380.29
717,2023-06-30 21:00:00,0.67,1.9958,3.0,0.118,403.23
718,2023-06-30 22:00:02,0.60,1.9972,3.0,0.118,406.66
719,2023-06-30 22:59:57,0.60,1.9986,3.0,0.156,404.31


# Step 4
## Modify surface inflow data

This step presents various way to modify the surface inflow data.

You can modify the data in the following ways:
- `Add a constant`
- `Multiply by a constant`
- `Set to a constant`
- `Add a linear trend`
- `Change depth`
- `Add an inflow`
- `Remove an inflow`
- `Change order of inflows`

### Add Inflow

Here you add a surface inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the new inflow
    - any positive integer not larger than the amount of existing deep inflows plus one, or `-1`
    - default is `-1` to append as the last inflow
- `values`: the values of the new inflow, can be:
    - any non-negative real number (can also be a fraction): to set a constant value in time
    - 1D array of non-negative real numbers (can also be a fraction) of the same length as the other inflows: for a varying inflow
    - default is `0`, constant in time
- `depth` : the depth of the new inflow
    - any real number
    - default is `0`

In [11]:
index = 1                                        # 👉 CUSTOMIZE the position of the new inflow
values = 2                                       # 👉 CUSTOMIZE the value of the new inflow
depth = 0                                        # 👉 CUSTOMIZE the depth of the new inflow

surf, surf_order = add_inflow(                   # modify the data
    surf, surf_order, index, values, depth)      # keep track of the modifications with surf_order
display(surf.round(4))                           # display the modified data

✓ Added inflow at (1)


,Datetime,0.0 (1)
0,2023-06-01 00:00:00,2.0
1,2023-06-01 01:00:02,2.0
2,2023-06-01 01:59:57,2.0
3,2023-06-01 03:00:00,2.0
4,2023-06-01 04:00:02,2.0
...,...,...
716,2023-06-30 19:59:57,2.0
717,2023-06-30 21:00:00,2.0
718,2023-06-30 22:00:02,2.0
719,2023-06-30 22:59:57,2.0


### Add constant

Here you add a constant to any surface inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `constant`: the constant to add
    - any real number (can also be negative and/or a fraction)
    - inflows should not become negative

In [12]:
index = 1                                        # 👉 CUSTOMIZE the inflow to operate on
constant = -1                                    # 👉 CUSTOMIZE the constant

surf.iloc[:, index] += constant                  # modify the data
if np.any(surf.iloc[:, index].values < 0):       # check for negative inflows
    surf.iloc[:, index] -= constant              # undo the modification
    print(f"Negative values for inflow ({index}) — no modification")
else:
    print(f"✓ Inflow ({index}) modified")
    display(surf.round(4))                       # display the modified data

✓ Inflow (1) modified


,Datetime,0.0 (1)
0,2023-06-01 00:00:00,1.0
1,2023-06-01 01:00:02,1.0
2,2023-06-01 01:59:57,1.0
3,2023-06-01 03:00:00,1.0
4,2023-06-01 04:00:02,1.0
...,...,...
716,2023-06-30 19:59:57,1.0
717,2023-06-30 21:00:00,1.0
718,2023-06-30 22:00:02,1.0
719,2023-06-30 22:59:57,1.0


### Multiply by constant

Here you multiply any surface inflow by a constant.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `constant`: the constant to multiply by
    - any non-negative real number (can also be a fraction)

In [13]:
index = 1                                        # 👉 CUSTOMIZE the inflow to operate on
constant = 1/5                                   # 👉 CUSTOMIZE the constant

if constant >= 0:                                # check that constant is non-negative
    surf.iloc[:, index] *= constant              # modify the data
    print(f"✓ Inflow ({index}) modified")
    display(surf.round(4))                       # display the modified data
else:
    print(f"Negative values for inflow ({index}) — no modification")

✓ Inflow (1) modified


,Datetime,0.0 (1)
0,2023-06-01 00:00:00,0.2
1,2023-06-01 01:00:02,0.2
2,2023-06-01 01:59:57,0.2
3,2023-06-01 03:00:00,0.2
4,2023-06-01 04:00:02,0.2
...,...,...
716,2023-06-30 19:59:57,0.2
717,2023-06-30 21:00:00,0.2
718,2023-06-30 22:00:02,0.2
719,2023-06-30 22:59:57,0.2


### Set to constant

Here you set any chosen surface inflow to a constant in time.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `constant`: the constant value of the inflow
    - any non-negative real number (can also be a fraction)

In [14]:
index = 1                                                   # 👉 CUSTOMIZE the inflow to operate on
constant = 4                                                # 👉 CUSTOMIZE the constant

if constant >= 0:                                           # check that constant is non-negative
    surf.iloc[:, index] = np.full((len(surf)), constant)    # modify the data
    print(f"✓ Inflow ({index}) modified")
    display(surf.round(4))                                  # display the modified data
else:
    print(f"Negative values for inflow ({index}) — no modification")

✓ Inflow (1) modified


,Datetime,0.0 (1)
0,2023-06-01 00:00:00,4.0
1,2023-06-01 01:00:02,4.0
2,2023-06-01 01:59:57,4.0
3,2023-06-01 03:00:00,4.0
4,2023-06-01 04:00:02,4.0
...,...,...
716,2023-06-30 19:59:57,4.0
717,2023-06-30 21:00:00,4.0
718,2023-06-30 22:00:02,4.0
719,2023-06-30 22:59:57,4.0


### Add linear trend

Here you add a linear trend to any chosen surface inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `step`: the time of change
    - any positive integer
    - `1` corresponds to one hour
    - accordingly `24` corresponds to one day
    -  `8760` corresponds to one year
- `slope`: the amount of change per time of change
    - any real number (can also be negative and/or a fraction)
    - inflow `(index)` increases by `slope` every `step` hours
- `offset`: the offset to the trend
    - any real number (can also be negative and/or a fraction)
    - default is `0`

In [15]:
index = 1                                                  # 👉 CUSTOMIZE the inflow to operate on
step = 24                                                  # 👉 CUSTOMIZE the time of change
slope = 1/15                                               # 👉 CUSTOMIZE the slope of the trend
offset = -2                                                # 👉 CUSTOMIZE the offset of the trend

rate = slope / step                                        # the rate of change
val_save = surf.iloc[:, index]                             # save the current values
surf.iloc[:, index] = (surf.iloc[:, index] +               # modify the data
    np.linspace(0, rate * (len(surf) - 1), len(surf)) +    # add the linear increase
    offset)                                                # add the offset
if np.any(surf.iloc[:, index].values < 0):                 # check for negative inflows
    deep.iloc[:, index] = val_save                         # undo the modification in case there are any negative inflows
    print(f"Negative values for inflow ({index}) — no modification")
else:
    print(f"✓ Inflow ({index}) modified")
    display(surf.round(4))                                 # display the modified data

✓ Inflow (1) modified


,Datetime,0.0 (1)
0,2023-06-01 00:00:00,2.0000
1,2023-06-01 01:00:02,2.0028
2,2023-06-01 01:59:57,2.0056
3,2023-06-01 03:00:00,2.0083
4,2023-06-01 04:00:02,2.0111
...,...,...
716,2023-06-30 19:59:57,3.9889
717,2023-06-30 21:00:00,3.9917
718,2023-06-30 22:00:02,3.9944
719,2023-06-30 22:59:57,3.9972


### Change depth

Here you change the depth of any chosen surface inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to modify
    - any integer with an existing deep inflow
- `depth`: the new depth
    - any real number (can also be negative and/or a fraction)

In [16]:
index = 1                                 # 👉 CUSTOMIZE the inflow to operate on
depth = -1                                # 👉 CUSTOMIZE the constant

surf = change_depth(surf, index, depth)   # modify the data
display(surf.round(4))                    # display the modified data

✓ Modified depth of inflow (1)


,Datetime,-1.0 (1)
0,2023-06-01 00:00:00,2.0000
1,2023-06-01 01:00:02,2.0028
2,2023-06-01 01:59:57,2.0056
3,2023-06-01 03:00:00,2.0083
4,2023-06-01 04:00:02,2.0111
...,...,...
716,2023-06-30 19:59:57,3.9889
717,2023-06-30 21:00:00,3.9917
718,2023-06-30 22:00:02,3.9944
719,2023-06-30 22:59:57,3.9972


#### Add inflows for next step

In [17]:
surf, surf_order = add_inflow(                    # modify the data
    surf, surf_order, 1, 2, 0)                    # keep track of the modifications with deep_order
surf, surf_order = add_inflow(                    # modify the data
    surf, surf_order, 3, 1, -1)                   # keep track of the modifications with deep_order
display(surf.round(4))                            # display the modified data

✓ Added inflow at (1)
✓ Added inflow at (3)


,Datetime,0.0 (1),-1.0 (2),-1.0 (3)
0,2023-06-01 00:00:00,2.0,2.0000,1.0
1,2023-06-01 01:00:02,2.0,2.0028,1.0
2,2023-06-01 01:59:57,2.0,2.0056,1.0
3,2023-06-01 03:00:00,2.0,2.0083,1.0
4,2023-06-01 04:00:02,2.0,2.0111,1.0
...,...,...,...,...
716,2023-06-30 19:59:57,2.0,3.9889,1.0
717,2023-06-30 21:00:00,2.0,3.9917,1.0
718,2023-06-30 22:00:02,2.0,3.9944,1.0
719,2023-06-30 22:59:57,2.0,3.9972,1.0


### Remove Inflow

Here you remove a surface inflow.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `index` : the index of the inflow to remove
    - any integer with an existing surface inflow, or `-1`
    - default is `-1` to remove the last inflow

In [18]:
index = -1                                       # 👉 CUSTOMIZE the inflow to operate on

surf, surf_order = remove_inflow(                # modify the data
    surf, surf_order, index)                     # keep track of the modifications with surf_order
display(surf.round(4))                           # display the modified data

✓ Removed inflow (3)


,Datetime,0.0 (1),-1.0 (2)
0,2023-06-01 00:00:00,2.0,2.0000
1,2023-06-01 01:00:02,2.0,2.0028
2,2023-06-01 01:59:57,2.0,2.0056
3,2023-06-01 03:00:00,2.0,2.0083
4,2023-06-01 04:00:02,2.0,2.0111
...,...,...,...
716,2023-06-30 19:59:57,2.0,3.9889
717,2023-06-30 21:00:00,2.0,3.9917
718,2023-06-30 22:00:02,2.0,3.9944
719,2023-06-30 22:59:57,2.0,3.9972


### Change order

Here you change the order of the surface inflows.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `order` : the new positions of the inflows
    - tuple with as many elements as existing surface inflows
    - example: `[2, 1]`: inflows (1) and (2) change position

In [19]:
order = [2, 1]                                   # 👉 CUSTOMIZE the new order

surf, surf_order = change_order(                 # modify the data
    surf, surf_order, order)                     # keep track of the modifications with surf_order
display(surf.round(4))                           # display the modified data

✓ Changed inflow order


,Datetime,-1.0 (1),0.0 (2)
0,2023-06-01 00:00:00,2.0000,2.0
1,2023-06-01 01:00:02,2.0028,2.0
2,2023-06-01 01:59:57,2.0056,2.0
3,2023-06-01 03:00:00,2.0083,2.0
4,2023-06-01 04:00:02,2.0111,2.0
...,...,...,...
716,2023-06-30 19:59:57,3.9889,2.0
717,2023-06-30 21:00:00,3.9917,2.0
718,2023-06-30 22:00:02,3.9944,2.0
719,2023-06-30 22:59:57,3.9972,2.0


# Step 5
## Save modified data

Here you save the modified inflow data to a new or the existing file.

**👉 CUSTOMIZE (next cell)** You can change this parameter:
- `copy` : Wheter to create a new inflow file (with _modified appended to the file_name) or to overwrite the original
    - `True` to create a new file (default)
    - `False` to overwrite the original

In [12]:
save_inflow_data(
    deep,                        # modified deep inflow data
    surf,                        # modified surface inflow data
    deep_order,                  # modified order of deep inflows
    surf_order,                  # modified order of surface inflows
    path = local_path,           # input path to the local file
    file = file_name,            # input name of the local file
    copy = True                  # 👉 CUSTOMIZE whether you want to copy and modify or overwrite the original
)

Writing modified data into: Biel\Qin_modified.dat
✓ Modified 721 rows
